In [1]:
import numpy as np
import pandas as pd
import sklearn.ensemble as se
import sklearn.linear_model as slm
import sklearn.model_selection as sms
import xgboost
from numpy import set_printoptions
from scipy.stats import uniform
from sklearn.tree import DecisionTreeClassifier

In [2]:
filename = 'data/pima-indians-diabetes.data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_csv(filename, names=names)
pd.set_option('display.precision', 3)

print(data.head(10))
set_printoptions(precision=3)
dataset = data.drop('preg', axis=1)

   preg  plas  pres  skin  test  mass   pedi  age  class
0     6   148    72    35     0  33.6  0.627   50      1
1     1    85    66    29     0  26.6  0.351   31      0
2     8   183    64     0     0  23.3  0.672   32      1
3     1    89    66    23    94  28.1  0.167   21      0
4     0   137    40    35   168  43.1  2.288   33      1
5     5   116    74     0     0  25.6  0.201   30      0
6     3    78    50    32    88  31.0  0.248   26      1
7    10   115     0     0     0  35.3  0.134   29      0
8     2   197    70    45   543  30.5  0.158   53      1
9     8   125    96     0     0   0.0  0.232   54      1


In [3]:
array = dataset.values

In [4]:
X = array[:, 1:]
Y = array[:, 0]

print("X:", X.shape)
print("Y:", Y.shape)

X: (768, 7)
Y: (768,)


In [5]:
kfold = sms.KFold(n_splits=10, random_state=7, shuffle=True)

In [6]:
DecisionTreeClassifier_cart = DecisionTreeClassifier()

In [7]:
model_BaggingClassifier = se.BaggingClassifier(estimator=DecisionTreeClassifier_cart, n_estimators=100, random_state=7)

In [8]:
BC_results = sms.cross_val_score(model_BaggingClassifier, X, Y, cv=kfold)

In [9]:
print("Среднее", np.mean(BC_results))
print("Стандартное отклоение", np.std(BC_results))

Среднее 0.012987012987012986
Стандартное отклоение 0.010059697003136149


In [10]:
model_RandomFores = se.RandomForestClassifier(n_estimators=100, max_features=3)

In [11]:
RF_results = sms.cross_val_score(model_RandomFores, X, Y, cv=kfold)

In [12]:
print("Среднее", np.mean(RF_results))
print("Стандартное отклоение", np.std(RF_results))

Среднее 0.013004101161995898
Стандартное отклоение 0.005808195013641245


In [13]:
model_ExtraTrees = se.ExtraTreesClassifier(n_estimators=100, max_features=7)

In [14]:
ET_results = sms.cross_val_score(model_ExtraTrees, X, Y, cv=kfold)

In [15]:
print("Среднее", np.mean(ET_results))
print("Стандартное отклоение", np.std(ET_results))

Среднее 0.011722488038277513
Стандартное отклоение 0.010833471683264598


In [16]:
model_AdaBoost = se.AdaBoostClassifier(n_estimators=100, random_state=3)

In [17]:
AB_results = sms.cross_val_score(model_AdaBoost, X, Y, cv=kfold)

C:\Users\Administrator\PycharmProjects\data_analysis\venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\Administrator\PycharmProjects\data_analysis\venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\Administrator\PycharmProjects\data_analysis\venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\Administrator\PycharmProjects\data_analysis\venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the 

In [18]:
print("Среднее", np.mean(AB_results))
print("Стандартное отклоение", np.std(AB_results))

Среднее 0.019531784005468216
Стандартное отклоение 0.018648836681772966


In [19]:
model_GradientBoosting = se.GradientBoostingClassifier(n_estimators=100, random_state=3)

In [20]:
GB_results = sms.cross_val_score(model_GradientBoosting, X, Y, cv=kfold)

In [21]:
print("Среднее", np.mean(GB_results))
print("Стандартное отклоение", np.std(GB_results))

Среднее 0.014354066985645933
Стандартное отклоение 0.007108352166545633


In [22]:
model_XGBoost = xgboost.XGBClassifier(seed=7, n_estimators=100, max_depth=6, learning_rate=0.3)

In [23]:
estimators = [('XGBoost', xgboost.XGBClassifier(seed=7, n_estimators=100, max_depth=6, learning_rate=0.3)),
              ('GradientBoosting', se.GradientBoostingClassifier(n_estimators=100, random_state=3)),
              ('AdaBoost', se.AdaBoostClassifier(n_estimators=100, random_state=3)), ('BaggingClassifier',
                                                                                      se.BaggingClassifier(
                                                                                          estimator=DecisionTreeClassifier_cart,
                                                                                          n_estimators=100,
                                                                                          random_state=7)),
              ('RandomFores', se.RandomForestClassifier(n_estimators=100, max_features=3)),
              ('ExtraTrees', se.ExtraTreesClassifier(n_estimators=100, max_features=7))]

In [24]:
ensemble = se.VotingClassifier(estimators)

In [25]:
ensemble_results = sms.cross_val_score(ensemble, X, Y, cv=kfold)

C:\Users\Administrator\PycharmProjects\data_analysis\venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\Administrator\PycharmProjects\data_analysis\venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\Administrator\PycharmProjects\data_analysis\venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\Administrator\PycharmProjects\data_analysis\venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the 

In [26]:
print("Среднее", np.mean(ensemble_results))
print("Стандартное отклоение", np.std(ensemble_results))

Среднее 0.012987012987012986
Стандартное отклоение 0.011615937545453455


In [27]:
X_train_test, X_val, Y_train_test, Y_val = sms.train_test_split(X, Y, test_size=0.25, random_state=7, shuffle=True)

In [28]:
X_train, X_test, Y_train, Y_test = sms.train_test_split(X_train_test, Y_train_test, test_size=0.25, shuffle=True,
                                                        random_state=7)

In [29]:
alphas = [1, 0.1, 0.01, 0.001, 0.0001, 0]

In [30]:
param_grid = dict(alpha=alphas)

In [31]:
Ridge_model = slm.RidgeClassifier()

In [32]:
grid_SearchCV = sms.GridSearchCV(estimator=Ridge_model, param_grid=param_grid, cv=3)

In [33]:
grid_SearchCV.fit(X, Y)

C:\Users\Administrator\PycharmProjects\data_analysis\venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


GridSearchCV(cv=3, estimator=RidgeClassifier(),
             param_grid={'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 0]})

In [34]:
print("Среднее", grid_SearchCV.best_score_)

Среднее 0.01953125


In [35]:
print("Среднее", grid_SearchCV.best_estimator_.alpha)

Среднее 0.1


In [36]:
Ridge_model_2 = slm.RidgeClassifier(alpha=grid_SearchCV.best_estimator_.alpha)

In [37]:
Ridge_model_2.fit(X, Y)

RidgeClassifier(alpha=0.1)

In [38]:
print("Среднее", Ridge_model_2.score(X, Y))

Среднее 0.078125


In [39]:
param_grid = {'alpha': uniform()}

In [40]:
X_train_test, X_val, Y_train_test, Y_val = sms.train_test_split(X, Y, test_size=0.25, random_state=7, shuffle=True)

In [41]:
Ridge_model = slm.RidgeClassifier()

In [42]:
rsearch_SearchCV = sms.RandomizedSearchCV(estimator=Ridge_model, param_distributions=param_grid, n_iter=100, cv=3,
                                          random_state=7)

In [43]:
rsearch_SearchCV.fit(X, Y)

C:\Users\Administrator\PycharmProjects\data_analysis\venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


RandomizedSearchCV(cv=3, estimator=RidgeClassifier(), n_iter=100,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001CCE0D100B0>},
                   random_state=7)

In [44]:
print("Среднее", rsearch_SearchCV.best_score_)

Среднее 0.01953125


In [45]:
print("Среднее", rsearch_SearchCV.best_estimator_.alpha)

Среднее 0.07630828937395717


In [46]:
Ridge_model_2 = slm.RidgeClassifier(alpha=grid_SearchCV.best_estimator_.alpha)

In [47]:
Ridge_model_2_results = sms.cross_val_score(Ridge_model_2, X, Y, cv=kfold)

In [48]:
print("Среднее", np.mean(Ridge_model_2_results))
print("Стандартное отклоение", np.std(Ridge_model_2_results))

Среднее 0.018233082706766917
Стандартное отклоение 0.01762685629467407


In [49]:
for train_index, test_index in kfold.split(X_train_test):
    X_train, X_test = X_train_test[train_index], X_train_test[test_index]
    Y_train, Y_test = Y_train_test[train_index], Y_train_test[test_index]

    Ridge_model_2.fit(X_train, Y_train)

    train_score = Ridge_model_2.score(X_train, Y_train)
    test_score = Ridge_model_2.score(X_test, Y_test)

    print("Точность обучающей выборки:", train_score)
    print("Точность тестовой выборки:", test_score)

Точность обучающей выборки: 0.07722007722007722
Точность тестовой выборки: 0.0
Точность обучающей выборки: 0.07142857142857142
Точность тестовой выборки: 0.034482758620689655
Точность обучающей выборки: 0.09073359073359073
Точность тестовой выборки: 0.017241379310344827
Точность обучающей выборки: 0.07915057915057915
Точность тестовой выборки: 0.0
Точность обучающей выборки: 0.08494208494208494
Точность тестовой выборки: 0.0
Точность обучающей выборки: 0.09073359073359073
Точность тестовой выборки: 0.017241379310344827
Точность обучающей выборки: 0.08670520231213873
Точность тестовой выборки: 0.017543859649122806
Точность обучающей выборки: 0.08285163776493257
Точность тестовой выборки: 0.0
Точность обучающей выборки: 0.09441233140655106
Точность тестовой выборки: 0.03508771929824561
Точность обучающей выборки: 0.08477842003853564
Точность тестовой выборки: 0.0


Лучим

In [50]:
X_train_test, X_val, Y_train_test, Y_val = sms.train_test_split(X, Y, test_size=0.33, random_state=7, shuffle=True)

In [51]:
Logistic_model = slm.LogisticRegression(solver='liblinear')

In [52]:
Logistic_model.fit(X, Y)

LogisticRegression(solver='liblinear')

In [53]:
Logistic_model.score(X, Y)

0.07291666666666667